### Code For Fetching Data

In [123]:
import quandl
from yahoofinancials import YahooFinancials
import requests
import pandas as pd
from pystore import store
from datetime import datetime
from hvplot import pandas
import copy

def add_close(data, tickers):
    terms = ['Value',
             'USD',
             'Hong Kong 14:00', 
             'Cash Price', 
             'Open',
            'Column 1'] + tickers
    for term in terms:
        if 'close' in data.columns:
            return data
        data.rename({term:'close'},axis=1,inplace=True)
    return data

def fetch(symbols, ticker_map, start, end, freq, collection, use_cache=True, verbose=0, **kwargs):
    """Return a key value dict of tickers and time series(dataframes)"""
    f = {}
    for symbol in symbols:
        try:
            if symbol in collection.items and use_cache:
                if verbose:
                    print("Retrieving Cache {}...".format(symbol))
                data = collection.item(symbol).to_pandas()
                assert('close' in data.columns)
            else:
                if verbose:
                    print("Fetching {}...".format(symbol))
                if symbol in ticker_map.keys():
                    source = ticker_map[symbol]['source']
                    ticker = ticker_map[symbol]['ticker']
                else:
                    source = 'yahoo'
                    ticker = symbol
                if source == 'yahoo':
                    yf = YahooFinancials(ticker)
                    data = yf.get_historical_price_data(start, end, freq)
                    data = pd.DataFrame(data[ticker]['prices'])
                elif source == 'quandl':
                    data = quandl.get(ticker, start_date=start,end_date=end,authtoken="pz51X6kcPpqth9s_J75f")
                    data = add_close(data, symbols)
                    data['date'] = pd.to_datetime(data.index)
                else:
                    print("Data Source {} not understood.".format(source))
                    raise Exception
                assert('close' in data.columns)
                collection.write(symbol,data,overwrite=True)
            f[symbol] = data
        except Exception as e:
            print("Failed to Fetch {}.".format(symbol))
            print("Data: {}".format(data))
            raise e 
    if verbose:
        print("Data Fetch Succesful.")
    return f

def format_date(sector):
    """Transforms timestamp column into pd.datetime column."""
    for symbol, df in sector.items():
        try:
            if not df['date'].dtype == 'datetime64[ns]':
                df['date'] = pd.to_datetime(df['date'].apply(datetime.fromtimestamp))
        except Exception as e:
            print("Failed to Normalize {}.".format(symbol))
            raise e 
    return sector

def fill(sector, start, end):
    """Ensure that all price series have the same index."""
    sector = {symbol:df.set_index('date').reindex(pd.date_range(start,end,freq='D'),method='nearest') for symbol, df in sector.items()}
    return sector
            
def norm(sector):
    """Adds a normalized price column to all series in sector."""
    for symbol, df in sector.items():
        try:
            df['normprice'] = df['close'] / df['close'].iloc[0]
        except Exception as e:
            print("Failed to Normalize {}.".format(symbol))
            raise e 
    return sector
        
def compare(sector, start, end):
    """Returns a new dataframe with all normalized prices"""
    sector = format_date(sector)
    sector = fill(sector, start, end)
    sector = norm(sector)
    sector = {symbol:df['normprice'] for symbol,df in sector.items()}
    return pd.DataFrame(sector)

### Define Sectors

In [124]:
indexes = {
    'SP500 (Large Cap)':{'source':'yahoo','ticker':'^GSPC',},
    'Russell 2000 (Small Cap)':{'source':'yahoo','ticker':'^RUT',},
    'BSE/SENSEX (INDIA)':{'source':'quandl','ticker':'BSE/SENSEX',},
    'FTSE 100 (London)':{'source':'yahoo','ticker':'^FTSE',},
    'CAC 40 (France)':{'source':'yahoo','ticker':'^GSPC',},
    'Nikkei 225 (TOKYO)':{'source':'yahoo','ticker':'^N225',},
    'Hang Seng (Hong Kong)':{'source':'yahoo','ticker':'^GSPC',},
    'Shanghai (China)':{'source':'yahoo','ticker':'^GSPC',},    
}

crypto = {
    'BTC':{'source':'yahoo','ticker':'BTC-USD',},
}

tech_stocks = {
    'AAPL':{'source':'yahoo','ticker':'AAPL'}, 
    'MSFT':{'source':'yahoo','ticker':'MSFT'}, 
    'INTC':{'source':'yahoo','ticker':'INTC'},
}

mutual_funds = {
    'USAGX':{'source':'yahoo','ticker':'USAGX',
        'name': 'USAA Precious Metals and Minerals Fund',}, 
}

precious_metals =  {
    'GOLD':{'source':'quandl','ticker':"WGC/GOLD_DAILY_USD",},
    'SILV':{'source':'quandl','ticker':"LBMA/SILVER",},
    'COPP':{'source':'quandl','ticker':"ODA/PCOPP_USD",},
    'PLAT':{'source':'quandl','ticker':"JOHNMATT/PLAT",},
    'PALL':{'source':'quandl','ticker':"JOHNMATT/PALL",},
    'IRID':{'source':'quandl','ticker':"JOHNMATT/IRID",},
}

agriculture = {
    'Barley':{'source':'quandl','ticker':'ODA/PBARL_USD'},
    'Corn':{'source':'quandl','ticker':'TFGRAIN/CORN.1'},
    'Rice':{'source':'quandl','ticker':'ODA/PRICENPQ_USD'},
    'Soybeans':{'source':'quandl','ticker':'TFGRAIN/SOYBEANS.1'},
    'Wheat':{'source':'quandl','ticker':'ODA/PWHEAMT_USD'},

    'Sugar':{'source':'quandl','ticker':'CHRIS/ICE_SB1.1'},
    'Coffee Robusta':{'source':'quandl','ticker':'ODA/PCOFFROB_USD'},
    'Cotton':{'source':'quandl','ticker':'CHRIS/ICE_CT1.1'},
    'Tea':{'source':'quandl','ticker':'ODA/PTEA_USD'},
    'Milk':{'source':'quandl','ticker':'COM/MILK'},
    
    'Bananas':{'source':'quandl','ticker':'ODA/PBANSOP_USD'},
    'Oranges':{'source':'quandl','ticker':'ODA/PORANG_USD'},
    'Peanuts':{'source':'quandl','ticker':'ODA/PGNUTS_USD'},
    
    'Olive Oil':{'source':'quandl','ticker':'ODA/POLVOIL_USD'},
    'Palm Oil':{'source':'quandl','ticker':'ODA/PPOIL_USD'},
    'Sunflower Oil':{'source':'quandl','ticker':'ODA/PSUNO_USD'},
    'Rapeseed Oil':{'source':'quandl','ticker':'ODA/PROIL_USD'},
}
materials = {
    'Rubber':{'source':'quandl','ticker':'ODA/PRUBB_USD'},
    'Soft Logs':{'source':'quandl','ticker':'ODA/PLOGORE_USD'},
    'Hard Logs':{'source':'quandl','ticker':'ODA/PLOGSK_USD'},
}

oil = {
    'IMF Crude':{'source':'quandl','ticker':"ODA/POILWTI_USD",},
    'Dubai Crude':{'source':'quandl','ticker':'OPEC/ORB'},
    'Natural Gas':{'source':'quandl','ticker':'CHRIS/CME_NG1.1'},
}

### Define Visualizations

In [125]:
ticker_map = {}
ticker_map.update(crypto)
ticker_map.update(precious_metals)
ticker_map.update(indexes)

collection = store('ltf').collection('kelowna')

demo1 = {
    'symbols':[     
        'GOLD',
        'SILVER',
        'BTC', 
        'COPP',
        'AAPL', 
        'GOOG',
        'SP500',
        ],
    'start':'2012-09-15',
    'end': '2019-09-15',
    'freq': 'daily',
    'ticker_map': ticker_map,
    'collection': collection,
    'title': "Normalized Asset Comparison"
}

demo2 = copy.deepcopy(demo1)
demo2.update({
    'symbols': list(precious_metals.keys()),
    'title':"Precious Metals"
})

demo3 = copy.deepcopy(demo1)
demo3.update({
    'symbols': list(indexes.keys()),
    'title':"Indexes"
})


tech = copy.deepcopy(demo1)
tech.update({
    'symbols': list(tech_stocks.keys()),
    'title':"Tech Stocks"
})

### Run a Demo

In [130]:
params = demo1
data = fetch(params['symbols'],
         params['ticker_map'],
         params['start'],
         params['end'],
         params['freq'],
         params['collection'],
         use_cache=True, 
         verbose=0,
            )
comp_start = '2017-09-15'
comp_end   = '2018-09-15'
comp = compare(data, comp_start, comp_end)

title = "{} {}:{}".format(params['title'],comp_start[:4],comp_end[:4])
comp.hvplot.line(logy=False, title=title)

:NdOverlay   [Variable]
   :Curve   [index]   (value)

### Run All Sectors

In [563]:
sectors = {
    'indexes':indexes,
    'crypto':crypto,
    'mutual_funds':mutual_funds,
    'precious_metals':precious_metals,
    'agriculture':agriculture,
    'materials':materials,
    'oil':oil,
}

sector_data = {}
for sector, tmap in sectors.items():
    ticker_map.update(tmap)
    symbols = list(tmap.keys())
    params = copy.deepcopy(demo1)
    params.update({
        'symbols':symbols,
    })
    data = fetch(**params, use_cache=True, verbose=0)
    sector_data[sector] = data

In [564]:
sector_data

{'indexes': {'SP500 (Large Cap)':                     date         high          low         open        close  \
                                                                                 
  0    2012-09-17 06:30:00  1465.630005  1457.550049  1465.420044  1461.189941   
  1    2012-09-18 06:30:00  1461.469971  1456.130005  1461.189941  1459.319946   
  2    2012-09-19 06:30:00  1465.150024  1457.880005  1459.500000  1461.050049   
  3    2012-09-20 06:30:00  1461.229980  1449.979980  1461.050049  1460.260010   
  4    2012-09-21 06:30:00  1467.069946  1459.510010  1460.339966  1460.150024   
  ...                  ...          ...          ...          ...          ...   
  1754 2019-09-09 06:30:00  2989.429932  2969.389893  2988.429932  2978.429932   
  1755 2019-09-10 06:30:00  2979.389893  2957.010010  2971.010010  2979.389893   
  1756 2019-09-11 06:30:00  3000.929932  2975.310059  2981.409912  3000.929932   
  1757 2019-09-12 06:30:00  3020.739990  3000.919922  3009.080078 